In [1]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import speclite.filters
from astropy.io import ascii

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs, effarea
import mejiro

In [2]:
filter_names = effarea._filter_name_list
filter = filter_names[0]

In [3]:
module_path = os.path.dirname(mejiro.__file__)
filter_responses_dir = os.path.join(module_path, 'data', 'filter_responses')
filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))

In [4]:
df_list = []

for f in filter_response_files:
    df_list.append(effarea.ecsv_to_pd(f))

In [6]:
sample_df = df_list[0]

In [8]:
df_list[0]

,wavelength,response
0,4730.0,0.000000
1,4740.0,0.000022
2,4750.0,0.000111
3,4760.0,0.000619
4,4770.0,0.002586
...,...,...
295,7680.0,0.001636
296,7690.0,0.000464
297,7700.0,0.000111
298,7710.0,0.000022


In [7]:
df_list[-1]

,wavelength,response
0,4710.0,0.000000
1,4720.0,0.000044
2,4730.0,0.000309
3,4740.0,0.001459
4,4750.0,0.005416
...,...,...
294,7650.0,0.001658
295,7660.0,0.000486
296,7670.0,0.000133
297,7680.0,0.000022


In [12]:
tot_df = sample_df.merge(df_list[-1], on='wavelength', how='outer', suffixes=(f'_SCA1', f'_SCA2'), sort=True)

tot_df.fillna(0)

,wavelength,response_SCA1,response_SCA2
0,4710.0,0.000000,0.000000
1,4720.0,0.000000,0.000044
2,4730.0,0.000000,0.000309
3,4740.0,0.000022,0.001459
4,4750.0,0.000111,0.005416
...,...,...,...
297,7680.0,0.001636,0.000022
298,7690.0,0.000464,0.000000
299,7700.0,0.000111,0.000000
300,7710.0,0.000022,0.000000


In [19]:
from copy import deepcopy
tot_df = deepcopy(df_list[0])

for sca, df in enumerate(df_list[1:]):
    tot_df = tot_df.merge(df, on='wavelength', how='outer', suffixes=(f'_SCA{str(sca + 1).zfill(2)}', f'_SCA{str(sca + 2).zfill(2)}'), sort=True)

tot_df = tot_df.fillna(0)

In [22]:
df_list[1]

,wavelength,response
0,4730.0,0.000000
1,4740.0,0.000022
2,4750.0,0.000133
3,4760.0,0.000707
4,4770.0,0.002874
...,...,...
295,7680.0,0.001127
296,7690.0,0.000309
297,7700.0,0.000066
298,7710.0,0.000022


In [20]:
tot_df

,wavelength,response_SCA01,response_SCA02,response_SCA03,response_SCA04,response_SCA05,response_SCA06,response_SCA07,response_SCA08,response_SCA09,response_SCA10,response_SCA11,response_SCA12,response_SCA13,response_SCA14,response_SCA15,response_SCA16,response_SCA17,response_SCA18
0,4710.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4720.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000044,0.000044,0.000044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000044,0.000044,0.000044
2,4730.0,0.000000,0.000000,0.000022,0.000022,0.000000,0.000066,0.000265,0.000221,0.000287,0.000000,0.000000,0.000022,0.000022,0.000022,0.000066,0.000265,0.000221,0.000309
3,4740.0,0.000022,0.000022,0.000133,0.000111,0.000066,0.000309,0.001216,0.000951,0.001348,0.000022,0.000022,0.000133,0.000111,0.000133,0.000332,0.001238,0.001039,0.001459
4,4750.0,0.000111,0.000133,0.000685,0.000553,0.000398,0.001437,0.004598,0.003537,0.005040,0.000133,0.000155,0.000663,0.000553,0.000597,0.001481,0.004686,0.003802,0.005416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,7680.0,0.001636,0.001127,0.000243,0.000398,0.000398,0.000199,0.000044,0.000022,0.000022,0.001680,0.001194,0.000221,0.000862,0.000376,0.000221,0.000044,0.000022,0.000022
298,7690.0,0.000464,0.000309,0.000044,0.000088,0.000088,0.000044,0.000000,0.000000,0.000000,0.000486,0.000332,0.000044,0.000243,0.000088,0.000044,0.000000,0.000000,0.000000
299,7700.0,0.000111,0.000066,0.000000,0.000022,0.000022,0.000000,0.000000,0.000000,0.000000,0.000111,0.000066,0.000000,0.000066,0.000022,0.000022,0.000000,0.000000,0.000000
300,7710.0,0.000022,0.000022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000022,0.000000,0.000022,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
for filter in tqdm(filter_names):
    filter_responses_dir = os.path.join(module_path, 'data', 'filter_responses')
    filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))

    df_list = []

    for f in filter_response_files:
        df_list.append(effarea.ecsv_to_pd(f))

    tot_df = deepcopy(df_list[0])

    for sca, df in enumerate(df_list[1:]):
        tot_df = tot_df.merge(df, on='wavelength', how='outer', suffixes=(f'_SCA{str(sca + 1).zfill(2)}', f'_SCA{str(sca + 2).zfill(2)}'), sort=True)

    tot_df = tot_df.fillna(0)

    avg = tot_df.iloc[:, 1:].mean(axis=1)
    tot_df.loc[:, "response"] = avg

    tot_df.to_csv(os.path.join(filter_responses_dir, f'{filter}_averaged.csv'), index=False)

100%|██████████| 11/11 [00:05<00:00,  1.89it/s]


In [32]:
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
import pandas as pd

for filter in filter_names:
    df = pd.read_csv(os.path.join(filter_responses_dir, f'{filter}_averaged.csv'))

    data = Table()
    data['wavelength'] = df['wavelength']
    data['wavelength'] *= u.Angstrom
    data['response'] = df['response']
    # data.write(os.path.join(filter_responses_dir, f'{filter}.ecsv'), overwrite=True)  

    speclite_filter = speclite.filters.FilterResponse(
                wavelength=data['wavelength'],
                response=data['response'],
                meta=dict(group_name=f'Roman', band_name=filter),
            )
    speclite_filter.save(filter_responses_dir)